In [77]:
# Import libraries
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup as bs
import re
import csv
from os.path import exists as e

In [78]:
def get_10_pages(base_url,url):
    pages_list = []
    pages_list.append(url)
    for i in range(9):
        html = urllib.request.urlopen(url)
        soup = bs(html, 'html.parser')
        next_page = soup.find('a', class_='btn-page next-page').get('href')
        pages_list.append(base_url + next_page)
        url = base_url + next_page
    return pages_list


In [79]:
def crawl_data(url='', topic=''):
#     if folder not in ['Khoa_hoc', 'Giai_tri', 'The_thao', 'Phap_luat', 'Giao_duc', 'Suc_khoe', 'Doi_song', 'Du_lich']:
#         print('folder must in [Khoa_hoc, Giai_tri, The_thao, Phap_luat, Giao_duc, Suc_khoe, Doi_song, Du_lich]')
#         return None
    # Get title and link VnExpress
    print('--------------------------------------------------------------')
    print('Getting title and link...')
    print('--------------------------------------------------------------')
    html = urllib.request.urlopen(url)
    soup = bs(html, 'html.parser')
    news = soup.find_all(re.compile('^h[1-9]{1}'), class_=re.compile('title.{1}news'))

    titles = []
    links = []
    for new in news:
        if new.find('a').get('title') != None and new.find('a').get('href') != None:
            print(new.find('a').get('title') + ' - Link: ' + new.find('a').get('href'))
            titles.append(new.find('a').get('title'))
            links.append(new.find('a').get('href'))
        
    # Read title content
    print('--------------------------------------------------------------')
    print('Getting title content...')
    print('--------------------------------------------------------------')
    contents = []
    video_news_idx = []
    # For link in links:
    for i in range(len(links)):
        html = urllib.request.urlopen(links[i])
        soup = bs(html, 'html.parser')
        description = soup.find('p', class_='description')
        if description != None: 
            if description.span != None: description.span.extract()
            content = description.text + ' '
        body = soup.find_all('p', class_='Normal')
        if len(body) < 2:
            video_news_idx.append(i)
            continue
        for para in body:
            if para.text not in content: content = content + re.sub('\s+', ' ', para.text) + ' '
        contents.append(content)

    for idx in sorted(video_news_idx, reverse=True):
        del links[idx]
        del titles[idx]
        
    # Save VnExpress 
    print('--------------------------------------------------------------')
    print('Saving to csv file...')
    print('--------------------------------------------------------------')
    file = open('VnExpress.csv', 'a+', encoding='UTF-8')
    file.seek(0)
    my_content = csv.reader(file, delimiter=',')
    my_title = []
    for row in my_content:
        if(len(row) == 0): continue
        else:
            my_title.append(row[0])
    file.seek(2)
    fwriter = csv.writer(file)
    if len(my_title) == 0: fwriter.writerow(['Title', 'Link', 'Content', 'Topic'])
#     if e('Khoa_hoc/VnExpress.csv'): 
#         file = open('Khoa_hoc/VnExpress.csv', 'w', encoding='UTF-8')
#         fwriter = csv.writer(file)
#         fwriter.writerow(['Title', 'Link', 'Content'])
#     else: 
#         file = open('Khoa_hoc/VnExpress.csv', 'a', encoding='UTF-8')
#         fwriter = csv.writer(file)
    for title, link, content in zip (titles, links, contents):
        if title not in my_title: 
            fwriter.writerow([title, link, content, topic])
    file.close()

In [80]:
# Get khoa hoc content
kh_page_list = get_10_pages('https://vnexpress.net', 'https://vnexpress.net/khoa-hoc')
for kh in kh_page_list:
    crawl_data(kh, 'Khoa_hoc')

--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Cá mập mako nhảy lên thuyền ngư dân - Link: https://vnexpress.net/ca-map-mako-nhay-len-thuyen-ngu-dan-4511430.html
Trung Quốc thử nghiệm thành công tên lửa tái sử dụng - Link: https://vnexpress.net/trung-quoc-thu-nghiem-thanh-cong-ten-lua-tai-su-dung-4511441.html
Cách dỗ trẻ nín khóc hiệu quả - Link: https://vnexpress.net/cach-do-tre-nin-khoc-hieu-qua-4510986.html
Mảnh vỡ tên lửa Trung Quốc sắp rơi xuống Philippines - Link: https://vnexpress.net/manh-vo-ten-lua-trung-quoc-sap-roi-xuong-philippines-4511434.html
100.000 gia súc nối đuôi nhau di cư - Link: https://vnexpress.net/100-000-gia-suc-noi-duoi-nhau-di-cu-4511446.html
Tọa đàm 'AI trong lĩnh vực tài chính' mở đầu Ngày hội AI4VN - Link: https://vnexpress.net/toa-dam-ai-trong-linh-vuc-tai-chinh-mo-dau-ngay-hoi-ai4vn-4510702.html
Xác ướp có thể thuộc về nữ hoàng Ai Cập cổ đại - Link: h

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Cấu trúc màu trắng kỳ lạ trên mặt hồ Mỹ - Link: https://vnexpress.net/cau-truc-mau-trang-ky-la-tren-mat-ho-my-4509101.html
Tàu Starship thử nghiệm tất cả động cơ - Link: https://vnexpress.net/tau-starship-thu-nghiem-tat-ca-dong-co-4509104.html
Cạnh tranh gắt gao về điểm số tại chung kết AI Hackathon 2022 - Link: https://vnexpress.net/canh-tranh-gat-gao-ve-diem-so-tai-chung-ket-ai-hackathon-2022-4509511.html
Top 6 AI Hackathon tập luyện trước chung kết - Link: https://vnexpress.net/top-6-ai-hackathon-tap-luyen-truoc-chung-ket-4509442.html
Trốn chó hoang, linh dương rơi vào hàm cá sấu - Link: https://vnexpress.net/tron-cho-hoang-linh-duong-roi-vao-ham-ca-sau-4508514.html
Xây dựng báo cáo hệ si

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Thước phim rõ nét nhất về xác tàu Titanic - Link: https://vnexpress.net/thuoc-phim-ro-net-nhat-ve-xac-tau-titanic-4507126.html
Kính viễn vọng James Webb chụp chuỗi vòng đồng tâm bí ẩn - Link: https://vnexpress.net/kinh-vien-vong-james-webb-chup-chuoi-vong-dong-tam-bi-an-4506985.html
Trung Quốc sắp hoàn thành đường sắt cao tốc vượt biển đầu tiên - Link: https://vnexpress.net/trung-quoc-sap-hoan-thanh-duong-sat-cao-toc-vuot-bien-dau-tien-4507073.html
Sứ mệnh Mặt Trăng Artemis 1 có thể hoãn nhiều tuần - Link: https://vnexpress.net/su-menh-mat-trang-artemis-1-co-the-hoan-nhieu-tuan-4506983.html
Tiềm năng khai thác tài nguyên quý từ tiểu hành tinh - Link: https://vnexpress.net/tiem-nang-khai-thac

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
NASA hủy phóng Sứ mệnh Mặt Trăng Artemis 1 - Link: https://vnexpress.net/nasa-huy-phong-su-menh-mat-trang-artemis-1-4505263.html
Loài cà rốt cao 5 m có thể gây mù vĩnh viễn - Link: https://vnexpress.net/loai-ca-rot-cao-5-m-co-the-gay-mu-vinh-vien-4505164.html
Đôi dép bằng vàng của vua Tutankhamun - Link: https://vnexpress.net/doi-dep-bang-vang-cua-vua-tutankhamun-4504936.html
Công nghệ in 3D đồ gỗ 'biến hình' - Link: https://vnexpress.net/cong-nghe-in-3d-do-go-bien-hinh-4504960.html
6 dự án được bình chọn cao nhất AI Awards 2022 - Link: https://vnexpress.net/6-du-an-duoc-binh-chon-cao-nhat-ai-awards-2022-4505036.html
Phát hiện tín hiệu từ tàu vũ trụ ở cách 23,3 tỷ km - Link: https://vnexpres

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Tiến sĩ Việt tại Google dùng AI chuyển văn bản thành ảnh - Link: https://vnexpress.net/tien-si-viet-tai-google-dung-ai-chuyen-van-ban-thanh-anh-4500864.html
Kỳ đà trèo lên cột điện trốn chó dữ - Link: https://vnexpress.net/ky-da-treo-len-cot-dien-tron-cho-du-4502709.html
Trung Quốc xây vòng tròn kính viễn vọng chu vi 3,14 km - Link: https://vnexpress.net/trung-quoc-xay-vong-tron-kinh-vien-vong-chu-vi-3-14-km-4502698.html
Thư viện nổi di động lớn nhất thế giới - Link: https://vnexpress.net/thu-vien-noi-di-dong-lon-nhat-the-gioi-4502705.html
Giải Tạ Quang Bửu đề xuất trao thêm lĩnh vực khoa học xã hội - Link: https://vnexpress.net/giai-ta-quang-buu-de-xuat-trao-them-linh-vuc-khoa-hoc-xa-hoi-45

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------


In [81]:
# Get giai tri content
gt_page_list = get_10_pages('https://vnexpress.net', 'https://vnexpress.net/giai-tri')
for gt in gt_page_list:
    crawl_data(gt, 'Giai_tri')

--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Bùi Quỳnh Hoa đoạt quán quân Siêu mẫu Quốc tế 2022 - Link: https://vnexpress.net/bui-quynh-hoa-doat-quan-quan-sieu-mau-quoc-te-2022-4511900.html
Dàn sao ủng hộ show drag queen của Thanh Duy Idol - Link: https://vnexpress.net/dan-sao-ung-ho-show-drag-queen-cua-thanh-duy-idol-4511877.html
Quang Lê: 'Nếu ham tiền, tôi đã cưới vợ đại gia' - Link: https://vnexpress.net/quang-le-neu-ham-tien-toi-da-cuoi-vo-dai-gia-4511469.html
Bóng hồng đóng cặp Chân Tử Đan trong 'Thiên Long Bát Bộ' - Link: https://vnexpress.net/bong-hong-dong-cap-chan-tu-dan-trong-thien-long-bat-bo-4511634.html
Britney Spears âm ỉ nỗi đau về gia đình - Link: https://vnexpress.net/britney-spears-am-i-noi-dau-ve-gia-dinh-4511078.html
Kịch 'Mùi của hạnh phúc': Lựa chọn một tình yêu - Link: https://vnexpress.net/kich-mui-cua-hanh-phuc-lua-chon-mot-tinh-yeu-4509417.html
Bà Michik

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Brad Pitt mặc phá cách trên thảm đỏ Venice - Link: https://vnexpress.net/brad-pitt-mac-pha-cach-tren-tham-do-venice-4509189.html
Phim về Marilyn Monroe được vỗ tay 14 phút - Link: https://vnexpress.net/phim-ve-marilyn-monroe-duoc-vo-tay-14-phut-4509135.html
'Nope': Phim kinh dị không kinh dị - Link: https://vnexpress.net/giai-tri/phim/thu-vien-phim/nope-483?fbclid=IwAR1Ex65ZGkRGdHHj5xE7ZZ5U8r6kka_0GsLo__tt8wDzdtiZlpUYZwh1EDk
Ngô Kiến Huy, Hoàng Yến Chibi đóng 'Trò chơi tử thần' - Link: https://vnexpress.net/ngo-kien-huy-hoang-yen-chibi-dong-tro-choi-tu-than-4509389.html
Chàng trai quê Trà Vinh thi Mister Global 2022 - Link: https://vnexpress.net/chang-trai-que-tra-vinh-thi-mister-global-2022

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Phim Việt tranh giải tại Liên hoan phim Busan - Link: https://vnexpress.net/phim-viet-tranh-giai-tai-lien-hoan-phim-busan-4507076.html
Triển lãm 'điêu khắc ánh sáng' của Bùi Văn Tự - Link: https://vnexpress.net/trien-lam-dieu-khac-anh-sang-cua-bui-van-tu-4506827.html
Phong độ tuổi 90 của Hồ Phong - Link: https://vnexpress.net/phong-do-tuoi-90-cua-ho-phong-4506970.html
Châu Kim Sang qua đời ở tuổi 26 - Link: https://vnexpress.net/chau-kim-sang-qua-doi-o-tuoi-26-4506880.html
Nhà hát kịch Hà Nội chờ xác minh thông tin về Hồng Đăng - Link: https://vnexpress.net/nha-hat-kich-ha-noi-cho-xac-minh-thong-tin-ve-hong-dang-4506941.html
'Có những niềm riêng' - tiếng lòng của Giang Hồng Ngọc - Link: http

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Dung nhan hoa hậu Hoàn vũ Ấn Độ 2022 - Link: https://vnexpress.net/dung-nhan-hoa-hau-hoan-vu-an-do-2022-4505099.html
Những bộ cánh nổi loạn ở VMAs - Link: https://vnexpress.net/nhung-bo-canh-noi-loan-o-vmas-4505034.html
Johnny Depp gây bất ngờ tại MTV VMAs - Link: https://vnexpress.net/johnny-depp-gay-bat-ngo-tai-mtv-vmas-4504987.html
Ưng Hoàng Phúc ra sách về 20 năm ca hát - Link: https://vnexpress.net/ung-hoang-phuc-ra-sach-ve-20-nam-ca-hat-4504922.html
Taylor Swift diện váy hở lưng ở VMAs - Link: https://vnexpress.net/taylor-swift-dien-vay-ho-lung-o-vmas-4504986.html
Trương Trí Lâm đón tuổi 51 bên vợ - Link: https://vnexpress.net/truong-tri-lam-don-tuoi-51-ben-vo-4504952.html
Cục trưởng Đ

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Rạp phim Việt xin được hoạt động sau 0h - Link: https://vnexpress.net/rap-phim-viet-xin-duoc-hoat-dong-sau-0h-4503111.html
Sao 'Ỷ Thiên Đồ Long Ký' được chồng chiều như công chúa - Link: https://vnexpress.net/sao-y-thien-do-long-ky-duoc-chong-chieu-nhu-cong-chua-4503364.html
Khánh Hà, Tô Chấn Phong làm tour xuyên Việt - Link: https://vnexpress.net/khanh-ha-to-chan-phong-lam-tour-xuyen-viet-4503156.html
Người thân quây quần ôn kỷ niệm về Phú Quang - Link: https://vnexpress.net/nguoi-than-quay-quan-on-ky-niem-ve-phu-quang-4503186.html
'Minions 2' thay cái kết khi ra rạp Trung Quốc - Link: https://vnexpress.net/minions-2-thay-cai-ket-khi-ra-rap-trung-quoc-4503224.html
Lâm Chí Dĩnh xuất viện - L

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------


In [82]:
# Get the thao content
tt_page_list = get_10_pages('https://vnexpress.net', 'https://vnexpress.net/the-thao')
for tt in tt_page_list:
    crawl_data(tt, 'The_thao')

--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Thế giới thể thao tri ân Federer - Link: https://vnexpress.net/the-gioi-the-thao-tri-an-federer-4511892.html
Federer giải nghệ - Link: https://vnexpress.net/federer-giai-nghe-4511886.html
Hoàng Đức không hội quân cùng tuyển Việt Nam - Link: https://vnexpress.net/hoang-duc-khong-hoi-quan-cung-tuyen-viet-nam-4511881.html
Cầu thủ Cần Thơ đình công - Link: https://vnexpress.net/cau-thu-can-tho-dinh-cong-4511854.html
VFF tìm cách để Quang Hải dự AFF Cup 2022 - Link: https://vnexpress.net/vff-tim-cach-de-quang-hai-du-aff-cup-2022-4511815.html
4 ứng viên xin rút khỏi đề cử Đại hội VFF khoá 9 - Link: https://vnexpress.net/4-ung-vien-xin-rut-khoi-de-cu-dai-hoi-vff-khoa-9-4511738.html
HLV Ten Hag chưa chắc xếp Ronaldo đá chính - Link: https://vnexpress.net/hlv-ten-hag-chua-chac-xep-ronaldo-da-chinh-4511683.html
VĐV Việt Nam có thể đảo ngược kết q

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Cựu cầu thủ Man Utd đá kiểu kung-fu vào mặt đối thủ - Link: https://vnexpress.net/cuu-cau-thu-man-utd-da-kieu-kung-fu-vao-mat-doi-thu-4510511.html
Dybala làm trái lời Mourinho - Link: https://vnexpress.net/dybala-lam-trai-loi-mourinho-4510574.html
Potter muốn làm nên lịch sử ở Chelsea - Link: https://vnexpress.net/potter-muon-lam-nen-lich-su-o-chelsea-4510099.html
Klopp: 'Thua Napoli 1-4 là kết quả tệ nhất ở Liverpool' - Link: https://vnexpress.net/klopp-thua-napoli-1-4-la-ket-qua-te-nhat-o-liverpool-4510506.html
Muller: 'Mane nhắc tôi không chuyền cho Lewandowski' - Link: https://vnexpress.net/muller-mane-nhac-toi-khong-chuyen-cho-lewandowski-4510498.html
Bayern lục đục nội bộ trước khi đấu

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Barca lập kỷ lục tăng giới hạn lương - Link: https://vnexpress.net/barca-lap-ky-luc-tang-gioi-han-luong-4509517.html
Son Heung-min chỉ biết cười sau chuỗi trận tịt ngòi - Link: https://vnexpress.net/son-heung-min-chi-biet-cuoi-sau-chuoi-tran-tit-ngoi-4509521.html
Vì sao Dortmund bán Haaland hè này? - Link: https://vnexpress.net/vi-sao-dortmund-ban-haaland-he-nay-4509509.html
Xavi không đổi Lewandowski lấy bất kỳ ai - Link: https://vnexpress.net/xavi-khong-doi-lewandowski-lay-bat-ky-ai-4509510.html
Chỉ có một ứng viên cho ghế Chủ tịch VFF - Link: https://vnexpress.net/chi-co-mot-ung-vien-cho-ghe-chu-tich-vff-4509675.html
Nhiều CLB Anh điêu đứng vì lịch đấu bị hoãn - Link: https://vnexpress.ne

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Bình Phước - Hà Nội: Đi theo tiếng gọi của HAGL? - Link: https://vnexpress.net/binh-phuoc-ha-noi-di-theo-tieng-goi-cua-hagl-4508717.html
Tiền đạo 72 triệu USD giúp Tottenham ra quân thắng lợi - Link: https://vnexpress.net/tien-dao-72-trieu-usd-giup-tottenham-ra-quan-thang-loi-4508613.html
Bayern lấy trọn ba điểm trên sân Inter - Link: https://vnexpress.net/bayern-lay-tron-ba-diem-tren-san-inter-4508609.html
TFG: 'Man Utd mua Antony đắt gấp đôi giá trị thực' - Link: https://vnexpress.net/tfg-man-utd-mua-antony-dat-gap-doi-gia-tri-thuc-4508396.html
HAGL loại Sài Gòn FC ở tứ kết Cup Quốc gia - Link: https://vnexpress.net/hagl-loai-sai-gon-fc-o-tu-ket-cup-quoc-gia-4508566.html
Spalletti: 'Liverp

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Roy Keane: 'Arteta không chịu khen Man Utd' - Link: https://vnexpress.net/roy-keane-arteta-khong-chiu-khen-man-utd-4507394.html
Lý do Haaland bỏ qua Real - Link: https://vnexpress.net/ly-do-haaland-bo-qua-real-4507495.html
Lý Hoàng Nam thắng tay vợt Thái Lan ở Bangkok Open 3 - Link: https://vnexpress.net/ly-hoang-nam-thang-tay-vot-thai-lan-o-bangkok-open-3-4507607.html
Eagle đáng giá 4 triệu USD của Dustin Johnson - Link: https://vnexpress.net/eagle-dang-gia-4-trieu-usd-cua-dustin-johnson-4507462.html
Tranh cãi về trọng tài ở trận Man Utd - Arsenal - Link: https://vnexpress.net/tranh-cai-ve-trong-tai-o-tran-man-utd-arsenal-4507372.html
Mourinho: 'Thua một trận 0-4 còn hơn bốn trận 0-1' - Lin

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------


In [83]:
# Get phap luat content
pl_page_list = get_10_pages('https://vnexpress.net', 'https://vnexpress.net/phap-luat')
for pl in pl_page_list:
    crawl_data(pl, 'Phap_luat')

--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Kiện Sở Giao thông Vận tải vì bị yêu cầu thi lại bằng lái ôtô - Link: https://vnexpress.net/kien-so-giao-thong-van-tai-vi-bi-yeu-cau-thi-lai-bang-lai-oto-4509946.html
Kẻ sát hại bố ruột lĩnh án chung thân - Link: https://vnexpress.net/ke-sat-hai-bo-ruot-linh-an-chung-than-4511870.html
Bị cáo giấu thi thể thiếu nữ: 'Gia đình họ xúc phạm tôi' - Link: https://vnexpress.net/bi-cao-giau-thi-the-thieu-nu-gia-dinh-ho-xuc-pham-toi-4511864.html
Dí roi điện vào em gái người yêu - Link: https://vnexpress.net/di-roi-dien-vao-em-gai-nguoi-yeu-4511680.html
Kẻ ám sát huyền thoại âm nhạc John Lennon bị từ chối ân xá lần thứ 12 - Link: https://vnexpress.net/ke-am-sat-huyen-thoai-am-nhac-john-lennon-bi-tu-choi-an-xa-lan-thu-12-4511640.html
Dùng xăng đốt chết vợ sau khi đọc tin nhắn - Link: https://vnexpress.net/dung-xang-dot-chet-vo-sau-khi-doc-tin-nhan-

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Cơn ghen tội lỗi của người chồng - Link: https://vnexpress.net/con-ghen-toi-loi-cua-nguoi-chong-4508910.html
Loạt tội ác ở nghĩa địa của đứa con sống trong đòn roi - Link: https://vnexpress.net/loat-toi-ac-o-nghia-dia-cua-dua-con-song-trong-don-roi-4508305.html
Áp lực nước tăng gây 'ngập' nhà dân, BQL chung cư có phải bồi thường? - Link: https://vnexpress.net/ap-luc-nuoc-tang-gay-ngap-nha-dan-bql-chung-cu-co-phai-boi-thuong-4509001.html
Hơn 10.000 người bị lừa vay tiền ngân hàng lãi suất thấp - Link: https://vnexpress.net/hon-10-000-nguoi-bi-lua-vay-tien-ngan-hang-khong-can-the-chap-4509034.html
Lập Facebook ảo vờ là thiếu nữ xinh đẹp để lừa đảo - Link: https://vnexpress.net/lap-facebook-ao-

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Âm mưu của nữ giúp việc tham lam - Link: https://vnexpress.net/am-muu-cua-nu-giup-viec-tham-lam-4506872.html
Có được trừ lương khi nhân viên làm sai? - Link: https://vnexpress.net/co-duoc-tru-luong-khi-nhan-vien-lam-sai-4506687.html
Mẹ ăn chay trường bị tù chung thân vì bỏ đói con đến chết - Link: https://vnexpress.net/me-an-chay-truong-bi-tu-chung-than-vi-bo-doi-con-den-chet-4506943.html
Câu đố: Kẻ nào cải trang để cướp ngân hàng? - Link: https://vnexpress.net/cau-do-ke-nao-cai-trang-de-cuop-ngan-hang-4506854.html
Bắt kẻ cướp túi xách người nước ngoài ở Hội An - Link: https://vnexpress.net/bat-ke-cuop-tui-xach-nguoi-nuoc-ngoai-o-hoi-an-4506877.html
Tội ác của kẻ giấu thi thể thiếu nữ - Link

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Uẩn khúc sau lời khai lỡ tay đánh chết kẻ trộm - Link: https://vnexpress.net/uan-khuc-sau-loi-khai-lo-tay-danh-chet-ke-trom-4504617.html
Câu đố: Chất độc được tẩm vào món ăn nào? - Link: https://vnexpress.net/cau-do-chat-doc-duoc-tam-vao-mon-an-nao-4504815.html
'Quái xế' trốn vào rừng khi bị cảnh sát vây bắt - Link: https://vnexpress.net/quai-xe-tron-vao-rung-khi-bi-canh-sat-vay-bat-4504801.html
Án mạng từ cuộc tình lầm lỡ với với nam phụ huynh - Link: https://vnexpress.net/an-mang-tu-cuoc-tinh-lam-lo-voi-voi-nam-phu-huynh-4504787.html
VKSND Cấp cao: Cần thu hồi tiền trụ trì đã trả nợ vì là tiền lừa đảo - Link: https://vnexpress.net/vksnd-cap-cao-can-thu-hoi-tien-tru-tri-da-tra-no-vi-la-tien

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
'Lao động bị lừa sang Campuchia rất khó giải cứu' - Link: https://vnexpress.net/cong-an-lao-dong-bi-lua-sang-campuchia-rat-kho-giai-cuu-4503062.html
Gần 3.000 người được xét đặc xá đợt 2/9 - Link: https://vnexpress.net/gan-3-000-nguoi-duoc-xet-dac-xa-dot-2-9-4503051.html
Câu đố: Giải mã vụ sát hại bạn thân trong quán bar - Link: https://vnexpress.net/cau-do-pha-vu-an-sat-hai-ban-than-sau-buoi-hop-mat-4503049.html
Con gái đại gia phủ nhận đồng phạm tham ô hơn 800 tỷ đồng - Link: https://vnexpress.net/con-gai-dai-gia-o-binhduong-phu-nhan-dong-pham-tham-o-hon-800-ty-dong-4502782.html
Bộ Công an đột kích nhiều 'lò' làm giấy tờ giả - Link: https://vnexpress.net/bo-cong-an-dot-kich-nhieu-lo-lam-gi

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------


In [84]:
# Get giao duc content
gd_page_list = get_10_pages('https://vnexpress.net', 'https://vnexpress.net/giao-duc')
for gd in gd_page_list:
    crawl_data(gd, 'Giao_duc')

--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Dưới 5 điểm mỗi môn vẫn trúng tuyển Đại học Tài nguyên và Môi trường TP HCM - Link: https://vnexpress.net/duoi-5-diem-moi-mon-van-trung-tuyen-dai-hoc-tai-nguyen-va-moi-truong-tp-hcm-4511862.html
Điểm chuẩn Đại học Kỹ thuật Y tế Hải Dương giảm - Link: https://vnexpress.net/diem-chuan-dai-hoc-ky-thuat-y-te-hai-duong-giam-4511885.html
Điểm chuẩn Đại học Giáo dục cao nhất 28,55 - Link: https://vnexpress.net/diem-chuan-dai-hoc-giao-duc-cao-nhat-28-55-4511852.html
Điểm chuẩn 9 trường, khoa thuộc Đại học Đà Nẵng - Link: https://vnexpress.net/diem-chuan-9-truong-khoa-thuoc-dai-hoc-da-nang-4511901.html
Điểm chuẩn Học viện An ninh nhân dân 2022 - Link: https://vnexpress.net/diem-chuan-hoc-vien-an-ninh-nhan-dan-2022-4511890.html
Điểm chuẩn Đại học Luật Hà Nội 2022 cao nhất 29,5 điểm - Link: https://vnexpress.net/diem-chuan-dai-hoc-luat-ha-noi-2022

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
10 đại học tốt nhất nước Mỹ năm 2022 - 2023 - Link: https://vnexpress.net/10-dai-hoc-tot-nhat-nuoc-my-nam-2022-2023-4510722.html
Kiểm tra IQ với bốn câu đố - Link: https://vnexpress.net/kiem-tra-iq-voi-bon-cau-do-4510842.html
Hai nữ sinh bị đánh hội đồng - Link: https://vnexpress.net/hai-nu-sinh-bi-danh-hoi-dong-4510709.html
Sáp nhập Đại học Nội vụ Hà Nội với Học viện Hành chính Quốc gia - Link: https://vnexpress.net/sap-nhap-dai-hoc-noi-vu-ha-noi-voi-hoc-vien-hanh-chinh-quoc-gia-4510686.html
Hiệu trưởng và hiệu phó phải đứng lớp vì thiếu giáo viên - Link: https://vnexpress.net/hieu-truong-va-hieu-pho-phai-dung-lop-vi-thieu-giao-vien-4510587.html
'Bật đèn xanh' cho thí sinh đăng ký lại phươn

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Nữ thạc sĩ 34 tuổi đạt 9.0 IELTS - Link: https://vnexpress.net/nu-thac-si-34-tuoi-dat-9-0-ielts-4508197.html
Hai học sinh bị mảng vữa trần phòng học rơi trúng - Link: https://vnexpress.net/hai-hoc-sinh-bi-mang-vua-tran-phong-hoc-roi-trung-4508166.html
Rải 52 đơn, chàng trai Hà Tĩnh giành học bổng tiến sĩ tại Bỉ - Link: https://vnexpress.net/rai-52-don-chang-trai-ha-tinh-gianh-hoc-bong-tien-si-tai-bi-4507863.html
Thích Sinh học nên chọn ngành nào? - Link: https://vnexpress.net/thich-sinh-hoc-nen-chon-nganh-nao-4507840.html
Hàng nghìn trẻ không được học bán trú vì vấn đề thu phí - Link: https://vnexpress.net/hang-nghin-tre-khong-duoc-hoc-ban-tru-vi-van-de-thu-phi-4508069.html
Dự kiến áp giá tr

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
TP HCM lo thiếu máy tính cho học sinh - Link: https://vnexpress.net/tp-hcm-lo-thieu-may-tinh-cho-hoc-sinh-4505696.html
Phân biệt âm /l/ và /r/ khi ở cuối từ tiếng Anh - Link: https://vnexpress.net/phan-biet-am-l-va-r-khi-o-cuoi-tu-tieng-anh-4505444.html
Thử thách đầu đời một mình tới Mỹ du học cấp 3 - Link: https://vnexpress.net/thu-thach-dau-doi-mot-minh-toi-my-du-hoc-cap-3-4503641.html
Hơn 30 tỷ đồng hỗ trợ thí sinh trong chương trình tiếp sức mùa thi - Link: https://vnexpress.net/hon-30-ty-dong-ho-tro-thi-sinh-trong-chuong-trinh-tiep-suc-mua-thi-4505523.html
Lương Sư phạm Văn có đủ sống nếu không dạy thêm? - Link: https://vnexpress.net/luong-su-pham-van-co-du-song-neu-khong-day-them-45052

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Hai quán quân của The Debate Challenge 2022 - Link: https://vnexpress.net/hai-quan-quan-cua-the-debate-challenge-2022-4502746.html
​Dạy trẻ theo phương pháp tích cực - Link: https://vnexpress.net/day-tre-theo-phuong-phap-tich-cuc-4501437.html
45.000 sinh viên từ 102 quốc gia đến Việt Nam du học - Link: https://vnexpress.net/45-000-sinh-vien-tu-102-quoc-gia-den-viet-nam-du-hoc-4502603.html
Trường đại học bị giả mạo thư trúng tuyển - Link: https://vnexpress.net/truong-dai-hoc-bi-gia-mao-thu-trung-tuyen-4502641.html
Thử thách IQ với bốn câu đố tư duy - Link: https://vnexpress.net/thu-thach-iq-voi-bon-cau-do-tu-duy-4502609.html
7 học sinh nhập viện sau khi hút thuốc lá điện tử - Link: https://vn

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------


In [85]:
# Get suc khoe content
sk_page_list = get_10_pages('https://vnexpress.net', 'https://vnexpress.net/suc-khoe')
for sk in sk_page_list:
    crawl_data(sk, 'Suc_khoe')

--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Thêm viện điều trị ung thư hiện đại tại TP HCM - Link: https://vnexpress.net/them-vien-dieu-tri-ung-thu-hien-dai-tai-tp-hcm-4511517.html
Anh cảnh báo biến chủng lạ của nCoV lây lan - Link: https://vnexpress.net/anh-canh-bao-bien-chung-la-cua-ncov-lay-lan-4511866.html
Người đầu tiên thế giới ghép tim từ mô sống - Link: https://vnexpress.net/nguoi-dau-tien-the-gioi-ghep-tim-tu-mo-song-4511847.html
Giải thoát cụ bà khỏi khối u che nửa mặt - Link: https://vnexpress.net/giai-thoat-cu-ba-khoi-khoi-u-che-nua-mat-4511735.html
Rước họa sau gói trị rạn da giá 8 triệu đồng - Link: https://vnexpress.net/ruoc-hoa-sau-goi-tri-ran-da-gia-8-trieu-dong-4511682.html
Cấp cứu thành công ca ung thư tiền liệt tuyến bí tiểu - Link: https://vnexpress.net/cap-cuu-thanh-cong-ca-ung-thu-tien-liet-tuyen-bi-tieu-4511499.html
17 triệu người tại châu Âu bị di chứng C

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Những cách chữa sùi mào gà hiệu quả - Link: https://vnexpress.net/nhung-cach-chua-sui-mao-ga-hieu-qua-4510311.html
Triệu chứng của các bệnh gan thường gặp - Link: https://vnexpress.net/trieu-chung-cua-cac-benh-gan-thuong-gap-4510243.html
Nguyên nhân gây chảy máu vùng kín sau giai đoạn mãn kinh - Link: https://vnexpress.net/nguyen-nhan-gay-chay-mau-vung-kin-sau-giai-doan-man-kinh-4509802.html
Tìm hiểu về tình trạng viêm nhiễm mũi họng - Link: https://vnexpress.net/tim-hieu-ve-tinh-trang-viem-nhiem-mui-hong-4509964.html
Giảm cân làm dịu cơn đau viêm khớp - Link: https://vnexpress.net/giam-can-lam-diu-con-dau-viem-khop-4510281.html
9 lời khuyên nuôi dạy trẻ mắc tăng động giảm chú ý - Link: http

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Nguyên nhân gây nhiễm trùng nấm Candida - Link: https://vnexpress.net/nguyen-nhan-gay-nhiem-trung-nam-candida-4508922.html
Những lưu ý khi chăm sóc người bệnh cắt bàng quang - Link: https://vnexpress.net/nhung-luu-y-khi-cham-soc-nguoi-benh-cat-bang-quang-4509434.html
4 loại thảo mộc giúp ngủ ngon - Link: https://vnexpress.net/4-loai-thao-moc-giup-ngu-ngon-4508697.html
Cà phê có nhiều chất chống oxy hóa hơn trái cây? - Link: https://vnexpress.net/ca-phe-co-nhieu-chat-chong-oxy-hoa-hon-trai-cay-4508576.html
Tập thể dục sai cách có thể gây hại xương khớp - Link: https://vnexpress.net/tap-the-duc-sai-cach-co-the-gay-hai-xuong-khop-4509377.html
Dấu hiệu trẻ thiếu hormone tăng trưởng - Link: https

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Thời điểm tốt để ngủ và thức dậy - Link: https://vnexpress.net/thoi-diem-tot-de-ngu-va-thuc-day-4507460.html
Điều trị và phòng ngừa suy giãn tĩnh mạch chân ở người tập gym - Link: https://vnexpress.net/dieu-tri-va-phong-ngua-suy-gian-tinh-mach-chan-o-nguoi-tap-gym-4508407.html
Tại sao vết thương của người bệnh đái tháo đường lâu lành? - Link: https://vnexpress.net/tai-sao-vet-thuong-cua-nguoi-benh-dai-thao-duong-lau-lanh-4508187.html
Trẻ bị viêm phổi, viêm phế quản có chữa khỏi hoàn toàn? - Link: https://vnexpress.net/tre-bi-viem-phoi-viem-phe-quan-co-chua-khoi-hoan-toan-4508427.html
8 món tráng miệng ít làm tăng đường huyết - Link: https://vnexpress.net/8-mon-trang-mieng-it-lam-tang-duong-h

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Vết mổ nứt toác khi sửa ngực ở spa - Link: https://vnexpress.net/vet-mo-nut-toac-khi-sua-nguc-o-spa-4507036.html
Argentina điều tra 4 ca tử vong vì viêm phổi lạ - Link: https://vnexpress.net/argentina-dieu-tra-4-ca-tu-vong-vi-viem-phoi-la-4507052.html
Vì sao Bộ Y tế trả lại 800 tỷ đồng đầu tư công? - Link: https://vnexpress.net/vi-sao-bo-y-te-tra-lai-800-ty-dong-dau-tu-cong-4506338.html
Khi nào dương vật ngừng phát triển? - Link: https://vnexpress.net/khi-nao-duong-vat-ngung-phat-trien-4507058.html
5 bài tập đi bộ ngoài trời giúp tăng thể lực - Link: https://vnexpress.net/5-bai-tap-di-bo-ngoai-troi-giup-tang-the-luc-4506099.html
5 dấu hiệu trái tim thay đổi trong thời kỳ mãn kinh - Link: htt

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------


In [86]:
# Get doi song content
ds_page_list = get_10_pages('https://vnexpress.net', 'https://vnexpress.net/doi-song')
for ds in ds_page_list:
    crawl_data(ds, 'Doi_song')

--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Bí quyết đồ xôi dẻo mềm, căng mẩy - Link: https://vnexpress.net/bi-quyet-do-xoi-deo-mem-cang-may-4511796.html
5 lời nhắc nhở giúp con thành công - Link: https://vnexpress.net/5-loi-nhac-nho-giup-con-thanh-cong-4511602.html
Trộm thận của vợ đi bán - Link: https://vnexpress.net/trom-than-cua-vo-di-ban-4511537.html
Con gái phải trả tiền phụng dưỡng cha mẹ dù bị bỏ rơi - Link: https://vnexpress.net/con-gai-phai-tra-tien-phung-duong-cha-me-du-bi-bo-roi-4511382.html
Biến nhà mặt tiền 2 m thành resort giữa lòng thành phố - Link: https://vnexpress.net/bien-nha-mat-tien-2-m-thanh-resort-giua-long-thanh-pho-4511079.html
Nỗi khổ con nhà giàu - Link: https://vnexpress.net/noi-kho-con-nha-giau-4510812.html
'Hai người mẹ' và cuộc chiến với ung thư của cậu bé mồ côi - Link: https://vnexpress.net/hai-nguoi-me-va-cuoc-chien-voi-ung-thu-cua-cau-be-mo-coi

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Đàn ông Hong Kong bị quấy rối tình dục trực tuyến nhiều hơn nữ - Link: https://vnexpress.net/dan-ong-hong-kong-bi-quay-roi-tinh-duc-truc-tuyen-nhieu-hon-nu-4507824.html
Những phòng trọ hộp diêm - Link: https://vnexpress.net/nhung-phong-tro-hop-diem-4507614.html
Ám ảnh nhà vệ sinh của học trò vùng cao - Link: https://vnexpress.net/am-anh-nha-ve-sinh-cua-hoc-tro-vung-cao-4503328.html
Ngôi nhà tường đá cao 2 m biệt lập trên núi cao - Link: https://video.vnexpress.net/tin-tuc/nhip-song/ngoi-nha-tuong-da-cao-2-m-biet-lap-tren-nui-cao-4507074.html
Cơ hội và thách thức chờ đón Việt Nam khi dân số già - Link: https://vnexpress.net/co-hoi-va-thach-thuc-cho-don-viet-nam-khi-dan-so-gia-4505638.html
Giả

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Người đàn ông mua cả ngôi làng đổ nát - Link: https://vnexpress.net/nguoi-dan-ong-mua-ca-ngoi-lang-do-nat-4503686.html
Những loại khí độc hại trong nhà - Link: https://vnexpress.net/nhung-loai-khi-doc-hai-trong-nha-4499919.html
Cuộc sống mới của hai cậu cháu mồ côi vì Covid - Link: https://vnexpress.net/cuoc-song-moi-cua-hai-cau-chau-mo-coi-vi-covid-4503102.html
Sử dụng men tiêu hóa và men vi sinh đúng cách - Link: https://vnexpress.net/su-dung-men-tieu-hoa-va-men-vi-sinh-dung-cach-4503601.html
Mẹo tránh đau nhức xương khớp khi đi du lịch - Link: https://vnexpress.net/meo-tranh-dau-nhuc-xuong-khop-khi-di-du-lich-4503352.html
Đậu phụ nhồi thịt sốt cà chua - Link: https://vnexpress.net/dau-phu

Làm gì khi nhà vệ sinh bốc mùi hôi? - Link: https://vnexpress.net/lam-gi-khi-nha-ve-sinh-boc-mui-hoi-4500961.html
Căn nhà hứng nắng mưa giữa lòng - Link: https://vnexpress.net/can-nha-hung-nang-mua-giua-long-4500566.html
Bạn đời vô tâm - Link: https://vnexpress.net/ban-doi-vo-tam-4499690.html
Mẹ và con trai cùng bị một người lừa tình - Link: https://vnexpress.net/me-va-con-trai-cung-bi-mot-nguoi-lua-tinh-4500906.html
Trốn đến nhà người dưng nhờ giúp ly hôn do bị bạo hành - Link: https://vnexpress.net/tron-den-nha-nguoi-dung-nho-giup-ly-hon-do-bi-bao-hanh-4497084.html
Cá nục kho thơm đậm vị hao cơm - Link: https://vnexpress.net/cach-lam-ca-nuc-kho-thom-dam-vi-hao-com-4500760.html
10 xu hướng trang trí nội thất dễ gây 'vỡ mộng' - Link: https://vnexpress.net/10-xu-huong-trang-tri-noi-that-de-gay-vo-mong-4500774.html
Người đàn ông 'dành cả đời để cưới vợ' - Link: https://vnexpress.net/nguoi-dan-ong-danh-ca-doi-de-cuoi-vo-4500673.html
Cách nói chuyện để trẻ chịu lắng nghe - Link: https://vn

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------


In [87]:
# Get du lich content
dl_page_list = get_10_pages('https://vnexpress.net', 'https://vnexpress.net/du-lich')
for dl in dl_page_list:
    crawl_data(dl, 'Du_lich')

--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Những bữa ăn ngon rẻ khi đi Nhật - Link: https://vnexpress.net/nhung-bua-an-ngon-re-khi-di-nhat-4501450.html
Hai khu phố đường tàu hút khách du lịch - Link: https://vnexpress.net/hai-khu-pho-duong-tau-hut-khach-du-lich-4511579.html
Quản lý khách sạn tiết lộ những thứ được mang về - Link: https://vnexpress.net/quan-ly-khach-san-tiet-lo-nhung-thu-duoc-mang-ve-4510898.html
Nơi hiếm hoi ở Singapore có thể xả rác thoải mái - Link: https://vnexpress.net/noi-hiem-hoi-o-singapore-co-the-xa-rac-thoai-mai-4511529.html
Mùa hồng Đà Lạt - Link: https://vnexpress.net/mua-hong-da-lat-4510856.html
Bộ ảnh 'Vẻ đẹp Việt Nam từ trên cao' đoạt giải quốc tế - Link: https://vnexpress.net/bo-anh-ve-dep-viet-nam-tu-tren-cao-doat-giai-quoc-te-4511393.html
6 món ngon không đụng hàng ở Phú Quốc - Link: https://vnexpress.net/6-mon-ngon-khong-dung-hang-o-phu-quoc-45

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Khách ăn phở bằng bát đá 300 độ C - Link: https://video.vnexpress.net/tin-tuc/food/khach-an-pho-bang-bat-da-300-do-c-4506335.html
Vì sao nhiều người chụp ảnh 'biển vô cực' không đẹp? - Link: https://vnexpress.net/vi-sao-nhieu-nguoi-chup-anh-bien-vo-cuc-khong-dep-4507942.html
7 ảnh 'Việt Nam từ trên cao' đoạt giải quốc tế - Link: https://vnexpress.net/7-anh-viet-nam-tu-tren-cao-doat-giai-quoc-te-4508247.html
Đến Cú Nhù San ngắm dải ngân hà - Link: https://vnexpress.net/den-cu-nhu-san-ngam-dai-ngan-ha-4507551.html
Các thủ tục khách Việt đi Nhật cần chuẩn bị - Link: https://vnexpress.net/cac-thu-tuc-khach-viet-di-nhat-can-chuan-bi-4508075.html
7 điểm đến lịch sử được yêu thích tại Việt Nam - Li

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Những điều khiến Bhutan thu hút du khách - Link: https://vnexpress.net/nhung-dieu-khien-bhutan-thu-hut-du-khach-4505507.html
Vietjet tung gần triệu vé giá ưu đãi dịp Quốc khánh - Link: https://vnexpress.net/vietjet-tung-gan-trieu-ve-gia-uu-dai-dip-quoc-khanh-4505876.html
Khách ngoại vào Việt Nam tăng nhanh - Link: https://vnexpress.net/khach-ngoai-vao-viet-nam-tang-nhanh-4505451.html
Mộc Châu được đề cử Giải thưởng Du lịch Thế giới 2022 - Link: https://vnexpress.net/moc-chau-duoc-de-cu-giai-thuong-du-lich-the-gioi-2022-4505734.html
Mặt tối những chuyến phượt bằng ôtô dài ngày - Link: https://vnexpress.net/mat-toi-nhung-chuyen-phuot-bang-oto-dai-ngay-4505568.html
Khách Việt mê vẻ bình yên làn

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Những món ăn lạ vị tại Nha Trang - Link: https://vnexpress.net/nhung-mon-an-la-vi-tai-nha-trang-4492810.html
Cắm trại như phim King Kong - Link: https://vnexpress.net/cam-trai-nhu-phim-king-kong-4502754.html
Khám phá Paris giống dân địa phương - Link: https://vnexpress.net/kham-pha-paris-giong-dan-dia-phuong-4502537.html
Bắt nhum và ăn tại chỗ ở quần đảo Hải Tặc - Link: https://vnexpress.net/bat-nhum-va-an-tai-cho-o-quan-dao-hai-tac-4502399.html
Việt Nam được giới thiệu là điểm đến sang trọng - Link: https://vnexpress.net/viet-nam-duoc-gioi-thieu-la-diem-den-sang-trong-4502773.html
Trải nghiệm du lịch khác lạ ở Nha Trang - Link: https://vnexpress.net/trai-nghiem-du-lich-khac-la-o-nha-trang-4

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Chinh phục núi lửa đang hoạt động ở Indonesia - Link: https://vnexpress.net/chinh-phuc-nui-lua-dang-hoat-dong-o-indonesia-4496949.html
Con người Lào gây ấn tượng với khách Việt - Link: https://vnexpress.net/con-nguoi-lao-gay-an-tuong-voi-khach-viet-4498331.html
Những đường cong của Đà Lạt - Link: https://vnexpress.net/nhung-duong-cong-cua-da-lat-4499323.html
Điểm lưu trú tiện ích giữa trung tâm thành phố Nha Trang - Link: https://vnexpress.net/diem-luu-tru-tien-ich-giua-trung-tam-thanh-pho-nha-trang-4498810.html
Nhiều du khách Việt bị huỷ vé máy bay đến đảo Jeju - Link: https://vnexpress.net/nhieu-du-khach-viet-bi-huy-ve-may-bay-den-dao-jeju-4499268.html
5 nơi lưu giữ xương cá voi ở Việt Nam

--------------------------------------------------------------
Saving to csv file...
--------------------------------------------------------------
--------------------------------------------------------------
Getting title and link...
--------------------------------------------------------------
Nơi nào đón bình minh trên đất liền sớm nhất Việt Nam? - Link: https://vnexpress.net/noi-nao-don-binh-minh-tren-dat-lien-som-nhat-viet-nam-4496483.html
Cây cầu cổ dáng thuyền nan úp ngược - Link: https://video.vnexpress.net/tin-tuc/du-lich/cay-cau-co-dang-thuyen-nan-up-nguoc-4495962.html
Cô gái Việt lái xe đi khắp Trung Quốc - Link: https://vnexpress.net/co-gai-viet-lai-xe-di-khap-trung-quoc-4495557.html
Phóng viên Mỹ mê nước mắm Phú Quốc - Link: https://vnexpress.net/phong-vien-my-me-nuoc-mam-phu-quoc-4496529.html
Đêm hè Paris dưới ống kính chàng trai Việt - Link: https://vnexpress.net/dem-he-paris-duoi-ong-kinh-chang-trai-viet-4495986.html
Tác phẩm thắng giải chủ đề 'Sự kiện' cuộc thi ảnh 